In [1]:
%pip install -U transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
# MODEL = "cardiffnlp/twitter-roberta-base-hate-latest"
# MODEL = "facebook/roberta-hate-speech-dynabench-r4-target"
MODEL = "irlab-udc/MetaHateBERT"

## Local Inference on GPU
Model page: https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [4]:
# Use a pipeline as a high-level helper - LOCAL INFERENCE
from transformers import pipeline

print("Loading BERT hate speech model locally...")
pipe = pipeline("text-classification", model=MODEL)

# Test examples
examples = [
    "Your mom is a fucking whore",
    "You are beautiful",
    "suck my dick you fucking bitch", 
    "all muslims are terrorists",
    "Women are terrible drivers",
    "I love spending time with my family",
    "Your mum is a beautiful whore that we love!"
]

print("\nRunning local inference on test examples:")
print("="*60)

for text in examples:
    result = pipe(text)
    # print(result)
    label = result[0]['label']
    confidence = result[0]['score']
    
    print(f"Text: {text[:50]:<50}")
    print(f"  Prediction: {label} (confidence: {confidence:.4f})")
    print("-" * 60)

Loading BERT hate speech model locally...


Device set to use cpu



Running local inference on test examples:
Text: Your mom is a fucking whore                       
  Prediction: LABEL_1 (confidence: 0.7952)
------------------------------------------------------------
Text: You are beautiful                                 
  Prediction: LABEL_0 (confidence: 0.9963)
------------------------------------------------------------
Text: suck my dick you fucking bitch                    
  Prediction: LABEL_1 (confidence: 0.9045)
------------------------------------------------------------
Text: all muslims are terrorists                        
  Prediction: LABEL_1 (confidence: 0.9956)
------------------------------------------------------------
Text: Women are terrible drivers                        
  Prediction: LABEL_1 (confidence: 0.9918)
------------------------------------------------------------
Text: I love spending time with my family               
  Prediction: LABEL_0 (confidence: 0.9979)
----------------------------------------------------

In [ ]:
# Load model directly for more control - LOCAL INFERENCE
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def predict_hate_speech(text):
    """
    Predict hate speech for a given text
    Returns: (label, confidence_score)
    """
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = F.softmax(outputs.logits, dim=-1)
    
    # Get predicted class and confidence
    predicted_class = torch.argmax(predictions, dim=-1).item()
    confidence = predictions[0][predicted_class].item()
    
    # Map class to label (check model config for actual mapping)
    label_map = {0: "NOT_HATE", 1: "HATE"}  # This might need adjustment based on model
    label = label_map.get(predicted_class, f"CLASS_{predicted_class}")
    
    return label, confidence

# Test the function
test_texts = [
    "I hate all immigrants",
    "This is a beautiful day",
    "You stupid idiot, go kill yourself",
    "Machine learning is fascinating"
]

print("\nManual inference results:")
print("="*60)

for text in test_texts:
    label, confidence = predict_hate_speech(text)
    print(f"Text: {text}")
    print(f"  Prediction: {label} (confidence: {confidence:.4f})")
    print("-" * 60)

Loading tokenizer and model...

Manual inference results:
Text: I hate all immigrants
  Prediction: HATE (confidence: 0.9997)
------------------------------------------------------------
Text: This is a beautiful day
  Prediction: NOT_HATE (confidence: 0.9998)
------------------------------------------------------------
Text: You stupid idiot, go kill yourself
  Prediction: NOT_HATE (confidence: 0.9991)
------------------------------------------------------------
Text: Machine learning is fascinating
  Prediction: NOT_HATE (confidence: 0.9999)
------------------------------------------------------------


In [ ]:
# Function to process your test data locally
import pandas as pd
import numpy as np

def process_test_data_locally(csv_path, output_path="submission.csv"):
    """
    Process test data using local BERT model and create submission file
    """
    print(f"Loading test data from {csv_path}...")
    
    # Load test data
    test_df = pd.read_csv(csv_path)
    print(f"Loaded {len(test_df)} test samples")
    
    # Initialize pipeline
    classifier = pipeline("text-classification", model=MODEL)
    
    predictions = []
    
    print("Running inference on test data...")
    for idx, row in test_df.iterrows():
        if idx % 100 == 0:
            print(f"Processed {idx}/{len(test_df)} samples...")
            
        text = str(row['post'])
        result = classifier(text)
        # result = model.predict(text)
        
        # Convert to binary (0=non-hate, 1=hate)
        # Adjust this mapping based on the actual model labels
        # print(result[0]['label'])
        # print(result)
        # if result['label'] == 'hate':
        if result[0]['label'] == 'LABEL_1':
            prediction = 1
        else:
            prediction = 0
            
        predictions.append(prediction)
    
    # Create submission file
    submission_df = pd.DataFrame({
        'row ID': test_df['id'],
        'label': predictions
    })
    
    submission_df.to_csv(output_path, index=False)
    print(f"\nSubmission file saved as: {output_path}")
    print(f"Predictions: {np.sum(predictions)} hate speech out of {len(predictions)} total")
    
    return submission_df

# Example usage (uncomment to run):
test_data_path = "../data/test.csv"  # Adjust path as needed
submission = process_test_data_locally(test_data_path)

Loading test data from data/test.csv...
Loaded 4296 test samples


Device set to use cpu
c:\Users\user\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Running inference on test data...
Processed 0/4296 samples...
Processed 100/4296 samples...
Processed 200/4296 samples...
Processed 300/4296 samples...
Processed 400/4296 samples...
Processed 500/4296 samples...
Processed 600/4296 samples...
Processed 700/4296 samples...
Processed 800/4296 samples...
Processed 900/4296 samples...
Processed 1000/4296 samples...
Processed 1100/4296 samples...
Processed 1200/4296 samples...
Processed 1300/4296 samples...
Processed 1400/4296 samples...
Processed 1500/4296 samples...
Processed 1600/4296 samples...
Processed 1700/4296 samples...
Processed 1800/4296 samples...
Processed 1900/4296 samples...
Processed 2000/4296 samples...
Processed 2100/4296 samples...
Processed 2200/4296 samples...
Processed 2300/4296 samples...
Processed 2400/4296 samples...
Processed 2500/4296 samples...
Processed 2600/4296 samples...
Processed 2700/4296 samples...
Processed 2800/4296 samples...
Processed 2900/4296 samples...
Processed 3000/4296 samples...
Processed 3100/42